In [2]:
import json
from typing import Dict
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END

In [3]:
load_dotenv()

True

In [4]:
search_query_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.7,
)   

researcher_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.7,
)

recommend_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.7,
)

In [5]:
# Define the state schema - the keeps track of the conversation history of all agents
class AppState(TypedDict): # Ensures that the state is of type Dict
    messages: list[BaseMessage] # ensures that the messages are of type BaseMessage

In [6]:
search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    )

# ai agents don't work well with tripadvisor

In [7]:
from datetime import datetime

def search_tool(state: Dict) -> Dict:
    messages = state["messages"]
    
    now = datetime.now()
    current_time = now.strftime("%I:%M %p")
    
    # First use LLM to refine the search query
    refine_messages = [
        SystemMessage(content="""
            You are a search query specialist.
            
            Your task is to reformulate a University of California Davis, CA student's food preferences into specific, 
            targeted search terms that will yield the most relevant food establishments.
                      
            Return only the search query, nothing else."""),

        HumanMessage(content=f"""
            Convert this request into a specific internet search query: 
            {messages[-1].content}""")
    ]

    refined_query = search_query_llm.invoke(refine_messages).content
    
    # Strip quotation marks from refined query
    refined_query = refined_query.replace('"', '')
    print(f"\n==== REFINED QUERY ====\n{refined_query}")
    
    # Use refined query with Tavily for search
    search_results = search.invoke({
        "query": refined_query,
    })
    
    # Format and add results to message history
    formatted_results = json.dumps(search_results, indent=2)
    messages.append(AIMessage(content=formatted_results))
    
    print(f"\n==== SEARCH TOOL RESULTS ====\n{formatted_results}")
    
    return {"messages": messages}

In [8]:
# Define the research node
def research(state: Dict) -> Dict:
    messages = state["messages"]
    research_messages = [
        SystemMessage(content='''
            You are an expert on food establishments for UC Davis students and research analysis using web content. Analyze the provided search results and extract optimal food establishment options along with detailed information about what they offer. 
            
            For each food establishment, please include:
            - **Establishment Name**: The name and location of the food establishment.
            - **Key Features**: What makes this place stand out (e.g., unique dishes, budget-friendly options, high-protein meals, late-night availability).
            - **Offerings & Specialties**: Summary of the menu, including specific dishes or meal types and who it’s best for (e.g., vegan options, quick bites, group dining).
            - **Pros & Cons**: Brief evaluation points, including food quality, service, ambiance, and suitability for specific student needs.
            - **Confirmed Hours**: Up-to-date operating hours, including if they are open late or during specific times (e.g., breakfast, lunch, dinner).
            - **Actionable Tips**: Recommendations for the best time to visit, must-try dishes, insider tips, and any available student discounts.
            
            Organize your response into clear sections for each food establishment. Use bullet points or headings where appropriate for clarity.
            Recommend only food establishments that are found in the search results.
        '''),
        *messages # Transfer conversation history
    ]
    response = researcher_llm.invoke(research_messages)
    print(f'\n=====RESEARCH AGENT NODE=====\n{response}')
    return {"messages": messages + [response]}
 
# 1. query 2. example 3. context (why are we doing this?) is recipe for a good promp

In [9]:
# Define the explain node
def recommend(state: Dict) -> Dict:
    messages = state["messages"]
    recommendation_messages = [
        SystemMessage(content='''
            You are an expert on food establishment selection for University of California, Davis students known for providing clear and professional recommendations.
            
            Based on the research findings provided, please perform the following tasks:
            1. Identify and rank the top food establishments that best meet the user's query.
            2. For each top destination, provide:
                - **Food Establishment Name**: The name and exact location of the food establishment.
                - **Why It's Ideal: **: A detailed explanation of why this food establishment is a great fit for the student's needs, including unique features, specific menu offerings, and suitability for particular preferences (e.g., budget-friendly, high-protein meals, vegan options).
                - **Key Features**: Highlight any standout attractions, special amenities (e.g., Wi-Fi, study-friendly spaces), and what makes it unique.
                - **Recommendations**: Actionable tips for planning a visit, including the best time to go, must-try dishes, insider advice, confirmed hours of operation, and any available student discounts.
                
            Organize your response in a clear, structured format (using headings or bullet points) to ensure it is easy to understand.
            Be as detailed and informative as necessary, as a food expert would be when providing a recommendation to a client.
        '''),
        *messages
    ]
    response = recommend_llm.invoke(recommendation_messages)
    print(f"\n=====FOOD EXPERT NODE====={response}")
    return {"messages": messages + [response]}

In [10]:
# Build the graph
graph = StateGraph(AppState)
graph.add_node("search", search_tool)
graph.add_node("research", research)
graph.add_node("recommend", recommend)

# Define the edges
graph.set_entry_point("search")
graph.add_edge("search", "research")
graph.add_edge("research", "recommend")
graph.add_edge("recommend", END)

In [11]:
# Define your function to run the graph
def run_conversation(user_input: str):
    """
    Runs the conversation graph with the given user input.
    
    Args:
        user_input (str): The user's input message.
    
    Returns:
        str: The final output message from the conversation.
    """
    initial_state = {
        "messages": [
            HumanMessage(content=user_input)
        ]
    }
    app = graph.compile()
    output = app.invoke(initial_state)
    return output["messages"][-1].content

result = run_conversation("I'm looking for yummy Asian food for dinner that's affordable and open til around 10pm")
print(result)


==== REFINED QUERY ====
yummy affordable Asian food open until 10pm Davis CA

==== SEARCH TOOL RESULTS ====
[
  {
    "url": "https://m.yelp.com/search?find_desc=asian+food+open+late&find_loc=Davis%2C+CA",
    "content": "Top 10 Best asian food open late Near Davis, California ; 1. MightyGoodFood. 5.0 (7 reviews) ; 2. Jusco Japanese Restaurant. 3.5 (387 reviews) ; 3. Teabo Caf\u00e9. 3.5"
  },
  {
    "url": "https://www.yelp.com/search?find_desc=Chinese+Food+Open+Late&find_loc=Davis%2C+CA",
    "content": "Top 10 Best Chinese Food Open Late in Davis, CA - February 2025 - Yelp - Oyama BBQ, Star KTV Lounge, Red 88 - Davis, Tea Hub, Gold Coast Cuisine,"
  },
  {
    "url": "https://wanderlog.com/list/geoCategory/471629/best-asian-food-in-davis",
    "content": "The 21 best Asian food in Davis \u00b7 1 Sophia's Thai Kitchen \u00b7 2 Thai Nakorn \u00b7 3 Wok of Flame \u987a\u5229\u996d\u5e97 \u00b7 4 Zen Toro Japanese Bistro \u00b7 5 Davis Sushi & Fusion \u00b7 6 Davis"
  },
  {
    "url"